# The following are analysis performed for this Bachelor Thesis

In [1]:
%load_ext autoreload
%autoreload 2

import header as head
import json
import os

annotations_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations"
community_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection"
bridges_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges"

There are 74 rumour source tweet with 0 non-rumour source tweet within the total of 74 rumour tweet.

In [ ]:
# p = head.getPath()
# print(p)
head.count()

In [ ]:
# merging following relationship into a JSON file
head.followRelate()

with open('/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json') as f:
    data = json.load(f)
    followRelate_Ids = list(data.keys())

In [ ]:
# generate dictionary for tweetId and userId conversion
# results are saved in Id-conversions folder
head.gen_idConversion()

There are followers who reacted to the source tweets. 
* reacts from followers are saved in `react_follows`
* reacts from non-followers are saved in `react_not_follows`

In [ ]:
# See whether there are any reactors that follows the person they responded to 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json") as f:
    followingRelate = json.load(f)

head.is_follower_react(followingRelate)

## Extracting all charlie hebdo annotations

In [ ]:
annotation_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/en-scheme-annotations.json"
head.extract_annotations("charliehebdo",annotation_path)

Organize the annotations by source tweets with their corresponding Direct Replies and Deep Replies.

In [ ]:
# Organize all the annotation based on source tweets
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/filtered_annotations.json") as f:
    data = json.load(f)

head.org_annotations(data)

For each source tweet, we extract all direct replies and deep replies which the respondents follow the user that they replied to.

In [ ]:
# Filter out reactions from followers from organized annotation
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/organized_annotations.json") as f:
    org_annots = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/react_follows.json") as f:
    react_follow = json.load(f)

for annot in list(org_annots):
    try: 
        reacts = react_follow[annot]
    
        for rep in (org_annots[annot])["Direct Replies"]:
            tweetId = rep["tweetid"]
            if not (tweetId in reacts): (org_annots[annot])["Direct Replies"].pop(rep)
        
        for rep in (org_annots[annot])["Deep Replies"]:
            tweetId = rep["tweetid"]
            if not (tweetId in reacts): (org_annots[annot])["Deep Replies"].pop(rep)
    
    except: 
        org_annots.pop(annot)

head.writeToJSON(head.getRoot(),"follower_annotations",org_annots)

We translate the follower_annotation data to refrence each source tweet by its userId instead of tweetId. 

The JSON file follows the following structure.
```
source tweet userId : {
    sourt tweetId :
    support :
    certainty :
    supportList : [
        certain : []
        Somewhat certain : []
        uncertain : []
    ]
    denyList: []
    commentList : []
}
```

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/Id-conversions/tweet_to_user.json") as f:
    tweet_to_user = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/follower_annotations.json") as f:
    follower_annotations = json.load(f)

dic = {}
for annot in follower_annotations:
    src_userId = tweet_to_user[annot]
    src_tweet = follower_annotations[annot]
    dic[src_userId] = {
        "source_tweetId" : annot,
        "support" : (src_tweet["Source Tweets"])["support"],
        "certainty" : (src_tweet["Source Tweets"])["certainty"],
        "supportList" : {
            "certain" : [],
            "somewhat-certain" : [],
            "uncertain" : []
        },
        "denyList": {
            "certain" : [],
            "somewhat-certain" : [],
            "uncertain" : []
        },
        "commentList" : []
    }
    cur_dic = dic[src_userId]
    dir_rep = src_tweet["Direct Replies"]
    for d in dir_rep:
        id = tweet_to_user[d["tweetid"]]
        if d["responsetype-vs-source"] == "comment":
            cur_dic["commentList"].append(id)

        elif d["responsetype-vs-source"] == "agreed":
            supporList = cur_dic["supportList"]
            if d["certainty"] == "certain":
                supporList["certain"].append(id)
            elif d["certainty"] == "somewhat-certain":
                supporList["somewhat-certain"].append(id)
            else :
                supporList["uncertain"].append(id)
        else:
            denyList = cur_dic["denyList"]
            if d["certainty"] == "certain":
                denyList["certain"].append(id)
            elif d["certainty"] == "somewhat-certain":
                denyList["somewhat-certain"].append(id)
            else :
                denyList["uncertain"].append(id)


head.writeToJSON("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations","userId_annotations",dic)


## Community Detection

We separate all the users into rumour and non-rumour communities based on keys of `support` and `certainty`. For the sake of this analysis, source tweets and reply tweets expresses same level value of user opinion. The categorization of a user depends on the number of tweets that support or deny a rumour. If there are same number of rumour and non-rumour tweets, we compare the number of certain tweets between rumour and non-rumour tweets. 

### **source tweets**  
We only consider rumour threads with "suporting" and "denying" values for their support key. In addition, their certainty as to be "certain" or "somewhat-certain". 

### **replies**  
The classification of Direct replies and Deep replies will be based on their `responsetype-vs-source` in comparison to its source tweet. Similar to source tweet classification, only certainties of "certain" and "somewhat-certain" will be considered. 

In [ ]:
# Further organize "filtered_annotation" file by using their tweetIds as keys.
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/filtered_annotations.json") as f:
    annots = json.load(f)

soruces = annots["Source Tweets"]
dir_rep = annots["Direct Replies"]
deep_rep = annots["Deep Replies"]

dic = {}
dic["Source_Tweets"] = {}
dic["Reply_Tweets"] = {}
src_dic = dic["Source_Tweets"]
rep_dic = dic["Reply_Tweets"]

for s in soruces:
    support = s["support"]
    if support != "underspecified":
        if s["certainty"] != "uncertain":
            src_dic[s["tweetid"]] = {
                "support": s["support"],
                "certainty" : s["certainty"]
            }


for d in dir_rep:
    respons_type = d["responsetype-vs-source"]
    if (respons_type != "comment" and respons_type != "appeal-for-more-information"):
        if d["certainty"] != "uncertain" and d["certainty"] != "underspecified":
            rep_dic[d["tweetid"]] = {
                "threadid" : d["threadid"],
                "responsetype-vs-source": d["responsetype-vs-source"],
                "certainty" : d["certainty"]
            }
        
for d in deep_rep:
    respons_type = d["responsetype-vs-source"]
    if (respons_type != "comment" and respons_type != "appeal-for-more-information"):
        if d["certainty"] != "uncertain" and d["certainty"] != "underspecified":
            rep_dic[d["tweetid"]] = {
                "threadid" : d["threadid"],
                "responsetype-vs-source": d["responsetype-vs-source"],
                "certainty" : d["certainty"]
            }

head.writeToJSON(annotations_path,"tweetId_annotations",dic)

In [ ]:
# Community Detection

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/tweetId_annotations.json") as f:
    annots = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/Id-conversions/tweet_to_user.json") as f:
    tweet_to_user = json.load(f)

sources = annots["Source_Tweets"]
replies = annots["Reply_Tweets"]
dic = {}

for s in sources:
    tweet = sources[s]
    
    try:
        user = dic[tweet_to_user[s]]
    except:
        dic[tweet_to_user[s]] = {
            "supporting" : {
                "certain" : [],
                "somewhat-certain" : []
            },
            "denying" : {
                "certain" : [],
                "somewhat-certain" : []
            }
        }
        user = dic[tweet_to_user[s]]

    if tweet["support"] == "supporting":
        user = user["supporting"]
    elif tweet["support"] == "denying":
        user = user["denying"]
    else:
        print("source: "+s)
        
    if tweet["certainty"] == "certain":
        user["certain"].append(s)
    elif tweet["certainty"] == "somewhat-certain":
        user["somewhat-certain"].append(s)

for r in replies:
    tweet = replies[r]

    # Check wheather the user responded to a valid source
    try:
        src_tweet = sources[tweet["threadid"]]
        src_support = src_tweet["support"]
    except:
        continue

    try:
        user = dic[tweet_to_user[r]]
    except:
        dic[tweet_to_user[r]] = {
            "supporting" : {
                "certain" : [],
                "somewhat-certain" : []
            },
            "denying" : {
                "certain" : [],
                "somewhat-certain" : []
            }
        }
        user = dic[tweet_to_user[r]]

    user_support = tweet["responsetype-vs-source"]
    
    if src_support == "supporting":
        if user_support == "agreed":
            user = user["supporting"]
        elif user_support == "disagreed":
            user = user["denying"]

    elif src_support == "denying":
        if user_support == "agreed":
            user = user["denying"]
        elif user_support == "disagreed":
            user = user["supporting"]
    
    else:
        print("rely: "+r)
    
    if tweet["certainty"] == "certain":
        user["certain"].append(s)
    elif tweet["certainty"] == "somewhat-certain":
        user["somewhat-certain"].append(s)
    else:
        print(r+" "+tweet["certainty"])
    


    
head.writeToJSON(annotations_path,"userId_annotations",dic)


In [11]:
# categorize all users into rumours and non-rumours
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/userId_annotations.json") as f:
    users = json.load(f)

communities = {
    "rumours" : [],
    "non_rumours" : []
}
rumours = communities["rumours"]
non_rumours = communities["non_rumours"]

for u in users:
    user = users[u]
    deny = user["denying"]
    support = user["supporting"]
    deny_size = len(deny["certain"]) + len(deny["somewhat-certain"])
    support_size = len(support["certain"]) + len(support["somewhat-certain"])
    if deny_size > support_size : non_rumours.append(u)
    elif deny_size < support_size : rumours.append(u)
    else:
        if len(deny["certain"]) > len(support["certain"]): non_rumours.append(u)
        elif len(deny["certain"]) < len(support["certain"]): rumours.append(u)
        else:
            if len(deny["somewhat-certain"]) > len(support["somewhat-certain"]) : non_rumours.append(u)
            elif len(deny["somewhat-certain"]) < len(support["somewhat-certain"]): rumours.append(u)
            else : print(u)
            
head.writeToJSON(community_path,"communities2",communities)


20159596
32272710
38233752
46363353


## Bridge Detection

1. Nodes that have followers from both communities. 
2. Nodes that follows other communites but only has follower from one commmunity

**code preparation**

In [ ]:
# Obtained who followed by whom relationship
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as f:
    follows = json.load(f)

who_followed_by_whom = {}
# Find who followed by whom
for user in follows:
    who_followed_by_whom[user] = []
    followed_by = who_followed_by_whom[user]

    for f in follows:
        if user in follows[f]: followed_by.append(f)

# head.writeToJSON(head.getRoot(),"who_followed_by_whom",who_followed_by_whom)

In [32]:
# obtain all followers from both communities per user 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as f:
    communities = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as f:
    follows = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who_followed_by_whom.json") as f:
    followed_by = json.load(f)

rumours_set = communities["rumours"]
non_rumours_set = communities["non-rumours"]

bridges = {}
for id in rumours_set:
    try:
        followers = followed_by[id]
    except: 
        continue
    
    bridges[id] = {
        "rumours" : [],
        "non_rumours" : []
    }
    node = bridges[id]

    for followee in followers:
        if followee in rumours_set: node["rumours"].append(followee)
        elif followee in non_rumours_set: node["non_rumours"].append(followee)
    
    if (len(node["rumours"])==0 and len(node["non_rumours"])==0): bridges.pop(id)

for id in non_rumours_set:
    try:
        followers = followed_by[id]
    except: 
        continue
    
    bridges[id] = {
        "rumours" : [],
        "non_rumours" : []
    }
    node = bridges[id]

    for followee in followers:
        if followee in rumours_set: node["rumours"].append(followee)
        elif followee in non_rumours_set: node["non_rumours"].append(followee)
    
    if (len(node["rumours"])==0 and len(node["non_rumours"])==0): bridges.pop(id)

# head.writeToJSON(bridges_path,"user_followed_by",bridges)

The following codes are for bridge detections

In [41]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as f:
    followers = json.load(f)

In [5]:
# A bridge has followers from both communities
two_com_bridges = {}
for id in followers:
   user = followers[id]
   if len(user["rumours"]) > 0 and len(user["non_rumours"]) > 0:
       two_com_bridges[id] = followers[id]

head.writeToJSON(bridges_path,"both_communities",two_com_bridges)

In [35]:
# A bridge only has followers from one community but follows other community 
one_com_bridges = {}
popList = []
for id in followers:
    user = followers[id]
    if len(user["rumours"]) == 0 or len(user["non_rumours"]) == 0:
        if len(user["rumours"]) == 0 and len(user["non_rumours"]) == 0:
            continue
        
        else : one_com_bridges[id] = followers[id]

for b in one_com_bridges:
    fans = one_com_bridges[b]
    fans["follow"]=[]
    try: followList = follows[b]
    except: continue

    if len(fans["rumours"]) == 0:
        for f in followList:
            if f in rumours_set: fans["follow"].append(f)
    else:
        for f in followList:
            if f in non_rumours_set: fans["follow"].append(f)
    print("here")
    print(str(len(fans["follow"])))
    if (len(fans["follow"])==0): popList.append(b)

for p in popList:
    print(p)
    one_com_bridges.pop(p)

head.writeToJSON(bridges_path,"single_community",one_com_bridges)

In [42]:
# A bridge only has followers from one community but follows other bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/both_communities.json") as f:
    bi_commun_bridges = json.load(f)
    bi_commun_bridges = list(bi_commun_bridges.keys())

one_com_bridges = {}
for id in followers:
    user = followers[id]
    if len(user["rumours"]) == 0 or len(user["non_rumours"]) == 0:
        if len(user["rumours"]) == 0 and len(user["non_rumours"]) == 0:
            continue
        
        else : one_com_bridges[id] = followers[id]

for b in list(one_com_bridges):
    fans = one_com_bridges[b]
    fans["followed_bridges"]=[]
    try: followList = follows[b]
    except: continue

    for f in bi_commun_bridges:
            if f in rumours_set: fans["followed_bridges"].append(f)
    if len(fans["followed_bridges"])==0: one_com_bridges.pop(b)

head.writeToJSON(bridges_path,"follows_bridges",one_com_bridges)

In [43]:
#  filter out the bridges ids from following list
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as f:
    following_relate = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/both_communities.json") as f:
    bi_commun_bridges = json.load(f)
    bi_commun_bridges = list(bi_commun_bridges.keys())

new_follow = {}

for id in following_relate:
    user = following_relate[id]
    new_follow[id] = {
        "rumours":[],
        "non_rumours":[],
        "bridges":[]
    }

    rumours = (new_follow[id])["rumours"]
    non_rumours = (new_follow[id])["non_rumours"]
    bridges = (new_follow[id])["bridges"]
    
    
    for r in user["rumours"]:
        if r in bi_commun_bridges: 
            bridges.append(r)
        else:
            rumours.append(r)
    
    for r in user["non_rumours"]:
        if r in bi_commun_bridges: 
            bridges.append(r)
        else:
            non_rumours.append(r)

head.writeToJSON(bridges_path,"followers",new_follow)

Differentiate bridges from other communities

In [2]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/both_communities.json") as f:
    bi_commun_bridges = json.load(f)
    bi_commun_bridges = list(bi_commun_bridges.keys())

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as f:
    communities = json.load(f)

communities["bridges"] = []

for id in communities["rumours"]:
    if (id == "428333") : print(id)
    if id in bi_commun_bridges : 
        (communities["rumours"]).remove(id)
        (communities["bridges"]).append(id)

for id in communities["non_rumours"]:
    if id in bi_commun_bridges : 
        (communities["non_rumours"]).remove(id)
        (communities["bridges"]).append(id)

# head.writeToJSON(community_path,"with_bridges3",communities)

## Centrality Analysis